# Discord Polygon

In [1]:
# Import required packages

from helper import *
import pandas as pd
import plotly.graph_objs as go
from polygon_ import *
from pprint import pprint

In [2]:
# Handle async use in Jupyter Notebook

import nest_asyncio
nest_asyncio.apply()

In [3]:
# Show supported timeframes

print(timeframes)
print("")
print(timespans)
print("")
print(patterns)

['4-Hour', '1-Day', '2-Day', '1-Week', '2-Week', '3-Week', '1-Month', '1-Quarter', '6-Month', '1-Year']

['minute', 'hour', 'day', 'week', 'month', 'quarter', 'year']

['doji', 'doubleinside', 'gapper', 'hammer', 'holygrail', 'inrev', 'inside', 'momohammer', 'nirvana', 'outrev', 'outside', 'po', 'rev', 'shooter']


In [4]:
# User input parameters

ticker = "VIV"
timeframe = "4-Hour"
pattern = "doubleinside"

In [5]:
# Get data

if timeframe not in timeframes:
    print(f"Error: timeframe={timeframe} not in {timeframes}")
interval, timespan = timeframe.split("-")
timespan = str(timespan).lower()
data = await fetch_stock_data(
    session=None, ticker=ticker, multiplier=interval, timespan=timespan, from_date=None, to_date=None, 
    adjusted=True, sort="desc", limit=5000
)
if "results" not in data:
    print(f"Error: data={data}")
results = data.get("results",[])
times1 = [pd.to_datetime(item.get("t",0), unit="ms") for item in results]
times1 # Before resample

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001B311265E90>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x000001B311260AD0>, 27879.25)]']
connector: <aiohttp.connector.TCPConnector object at 0x000001B311284B50>


[Timestamp('2025-03-14 21:30:00'),
 Timestamp('2025-03-14 19:30:00'),
 Timestamp('2025-03-14 19:00:00'),
 Timestamp('2025-03-14 18:30:00'),
 Timestamp('2025-03-14 18:00:00'),
 Timestamp('2025-03-14 17:30:00'),
 Timestamp('2025-03-14 17:00:00'),
 Timestamp('2025-03-14 16:30:00'),
 Timestamp('2025-03-14 16:00:00'),
 Timestamp('2025-03-14 15:30:00'),
 Timestamp('2025-03-14 15:00:00'),
 Timestamp('2025-03-14 14:30:00'),
 Timestamp('2025-03-14 14:00:00'),
 Timestamp('2025-03-14 13:30:00'),
 Timestamp('2025-03-14 13:00:00')]

In [6]:
# Timeframe resample

if timeframe in timeframes_resampled:
    results2 = resample_results(results, new_timeframe=timeframe)
else:
    results2 = results
times2 = [pd.to_datetime(item.get("t",0), unit="ms") for item in results2]
times2 # After resample

[Timestamp('2025-03-14 17:30:00'), Timestamp('2025-03-14 13:30:00')]

In [7]:
# Pattern check

pattern_func_name = f"is_{pattern}"
pattern_func = globals().get(pattern_func_name)
if not pattern_func:
    print(f"Error: Could not find {pattern_func_name} in globals")
if pattern not in pattern_candles_dict:
    print(f"Error: pattern={pattern}, not found in {pattern_candles_dict}")
candles_nbr = pattern_candles_dict.get(pattern,1)
results2 = results2[:candles_nbr]
pprint(results2)
has_pattern = pattern_func(*results2)
pt = "DO" if has_pattern else "DO NOT"
print("")
print(f"The {candles_nbr} most recent candles for ticker {ticker} on the {timeframe} timeframe {pt} have the {pattern} pattern")

[{'c': 8.84,
  'h': 8.915,
  'l': 8.83,
  'n': 6269.0,
  'o': 8.91,
  't': 1741973400000.0,
  'v': 584324.0,
  'vw': 8.858},
 {'c': 8.905,
  'h': 8.93,
  'l': 8.66,
  'n': 9873.0,
  'o': 8.7,
  't': 1741959000000.0,
  'v': 1033590.0,
  'vw': 8.9113}]


TypeError: is_doubleinside() missing 1 required positional argument: 'candle_3'

In [ ]:
# Create dataframe (before resample)

df = pd.DataFrame(data=results)
df["datetime"] = pd.to_datetime(df["t"], unit="ms")
df = df.sort_values(by="datetime", ascending=True, ignore_index=True) # Must be in ascending order for chart
df

In [ ]:
# Create dataframe (after resample)

df = pd.DataFrame(data=results2)
df["datetime"] = pd.to_datetime(df["t"], unit="ms")
df = df.sort_values(by="datetime", ascending=True, ignore_index=True) # Must be in ascending order for chart
df

In [ ]:
# Visual check

tick_interval = max(1, len(df) // 10)  # Show labels for approximately 10 points
tickvals = df["datetime"][::tick_interval]  # Select every nth value
ticktext = [pd.to_datetime(t).strftime('%H:%M') for t in tickvals]

trace1 = go.Candlestick(
    x = df["datetime"],
    open = df["o"],
    high = df["h"],
    low = df["l"],
    close = df["c"],
    name = "candles",
    increasing = dict(
        line = dict(
            color = '#9FFE36'
        ),
        fillcolor = '#9FFE36'
    ),
    decreasing = dict(
        line = dict(
            color = '#AF121A'
        ), 
        fillcolor = '#AF121A'
    ),
    customdata = df["datetime"],  # Pass the formatted datetime to customdata
    hoverinfo = "text",  # Enable custom text for hover
    text = (
        "Date/Time: " + df["datetime"].astype(str) + "<br>" +
        "Open: " + df["o"].astype(str) + "<br>" +
        "High: " + df["h"].astype(str) + "<br>" +
        "Low: " + df["l"].astype(str) + "<br>" +
        "Close: " + df["c"].astype(str)
    )
)

traces = [trace1]

# Create layout

layout = dict(
    title = f'Candle Bars for symbol {ticker} on {timeframe} timeframe',
    plot_bgcolor = '#000000',  # Set plot area background to black
    paper_bgcolor = '#a9a9a9',  # Set overall figure background to dark gray
    xaxis = dict(
        title = 'Datetime', # Datetime
        rangeslider_visible = False,
        type = "date", # ["date","category"]
        showgrid = False,
#         tickvals = tickvals,
#         ticktext = ticktext,
        # tickformat = "%H:%M",
        tickangle = 45,
    ),
    yaxis = dict(
        title = 'Price',
        tickformat = "$,.2f",
        showgrid = False,
    )
)

# Create figure

fig = go.Figure(
    data = traces,
    layout = layout
)

# Show figure

fig.show()

In [ ]:
# END